In [12]:
import glob 
import numpy as np
from functions import dice_coef_lung_sem_epsilon, dice_coef_lung, calcula_metricas
from utils import load_images_array
from skimage.io import imread, imshow

import matplotlib.pyplot as plt
import matplotlib.image as img 

In [13]:
def gabarito_2(pred, gt, value_mask=255):
    a = sum(pred==value_mask).sum()
    print(f'a: {a}')
    b = sum(gt==value_mask).sum()
    print(f'b: {b}')
    inter=0
    for i in range(pred.shape[0]):
        for j in range(pred.shape[1]):
            if pred[i,j]==gt[i,j] and pred[i,j]==value_mask:
                inter +=1
    print(f'inter: {inter}')
    dice = 2*inter/(a+b)
    return dice

In [14]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras import *
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam #não roda sem o tensorflow.keras
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.callbacks import LearningRateScheduler, ModelCheckpoint


### Método de importação do Lung

In [15]:
#Dados de treino e validação
IMAGE_LIB = './imgs_testes_metricas/teste_lung_2022/gt/'
MASK_LIB = './imgs_testes_metricas/teste_lung_2022/pred/'

IMG_HEIGHT, IMG_WIDTH = 4, 4 

all_images = [x for x in sorted(os.listdir(IMAGE_LIB)) if x[-4:] == '.png']

x_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(IMAGE_LIB + name, cv2.IMREAD_UNCHANGED).astype("int16").astype('float32') #IMREAD_UNCHANGED: manutenção do canal alfa
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4) #Interpolação de Lanczos
    im = (im - np.min(im)) / (np.max(im) - np.min(im)) #Transformando a escala dos pixels para valores entre 0 e 1
    x_data[i] = im


y_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(MASK_LIB + name, cv2.IMREAD_UNCHANGED).astype('float32')/255.
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST) #Interpolação por nearest-neighbor
    y_data[i] = im

### Imagens-Teste

In [16]:
TEMPO = []
ORIGINAL_SIZE = 5
NEW_SIZE = 4

In [17]:
# VERSÃO SEGMENTAÇÃO PULMÃO
# não deu certo com o imread
dice = dice_coef_lung(x_data, y_data)
dice = str(dice).split(',')[0].split('(')[1]
print(f'Coeficiente Dice (Versão usada com as tomografias de pulmão): {dice}')

Coeficiente Dice (Versão usada com as tomografias de pulmão): 0.9346041


In [18]:
# VERSÃO SEGMENTAÇÃO PULMÃO
# não deu certo com o imread
dice = dice_coef_lung_sem_epsilon(x_data, y_data)
dice = str(dice).split(',')[0].split('(')[1]
print(f'Coeficiente Dice (Versão usada com as tomografias de pulmão): {dice}')

Coeficiente Dice (Versão usada com as tomografias de pulmão): 0.9346041


#### Gabarito

In [19]:
TEMPO = []
ORIGINAL_SIZE = 5
NEW_SIZE = 4

# Choose train folder TM40 ou TM46
_folder = './imgs_testes_metricas/imgs_nv' 

GT_imgs = sorted(glob.glob(_folder + '/gt/*')) 
pred_imgs = sorted(glob.glob(_folder + '/pred/*'))

gt = load_images_array(GT_imgs, new_size = NEW_SIZE)
pred = load_images_array(pred_imgs, new_size = NEW_SIZE)

In [20]:
# GABARITO
dice_list = []
dice_list = calcula_metricas(gt, pred)  

#estatística
print(f'Dice list: {dice_list}')
mean = round(np.mean(dice_list), 4)
print(f'Coeficiente Dice (Versão do Laboratório): {mean}')

Dice list: [0.9600000000000001, 0.962962962962963]
Coeficiente Dice (Versão do Laboratório): 0.9615


### Imagem Pulmão

In [21]:
#Dados de treino e validação
IMAGE_LIB = './imgs_testes_metricas/teste_lung_2022/conjunto_a/'
MASK_LIB = './imgs_testes_metricas/teste_lung_2022/conjunto_b/'

IMG_HEIGHT, IMG_WIDTH = 512, 512

all_images = [x for x in sorted(os.listdir(IMAGE_LIB)) if x[-4:] == '.tif']

x_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(IMAGE_LIB + name, cv2.IMREAD_UNCHANGED).astype("int16").astype('float32') #IMREAD_UNCHANGED: manutenção do canal alfa
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4) #Interpolação de Lanczos
    im = (im - np.min(im)) / (np.max(im) - np.min(im)) #Transformando a escala dos pixels para valores entre 0 e 1
    x_data[i] = im


y_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH), dtype='float32')
for i, name in enumerate(all_images):
    im = cv2.imread(MASK_LIB + name, cv2.IMREAD_UNCHANGED).astype('float32')/255.
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST) #Interpolação por nearest-neighbor
    y_data[i] = im

In [22]:
# VERSÃO SEGMENTAÇÃO PULMÃO
# não deu certo com o imread
dice = dice_coef_lung(x_data, y_data)
dice = str(dice).split(',')[0].split('(')[1]
print(f'Coeficiente Dice (Versão usada com as tomografias de pulmão): {dice}')

Coeficiente Dice (Versão usada com as tomografias de pulmão): 0.7348611


In [23]:
# VERSÃO SEGMENTAÇÃO PULMÃO
# não deu certo com o imread
dice = dice_coef_lung_sem_epsilon(x_data, y_data)
dice = str(dice).split(',')[0].split('(')[1]
print(f'Coeficiente Dice (Versão usada com as tomografias de pulmão): {dice}')

Coeficiente Dice (Versão usada com as tomografias de pulmão): 0.7348611
